# *Covid-19 Tweets*

### *Loading Libraries*

In [156]:
# Loading Libraries

import pandas as pd
import numpy as np 

import requests, zipfile, io
import re
import csv
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

## Data Wrangling

**Data Wrangling has three steps:** 
- Gather Data

- Access Data

- Clean Data has three steps: 
    - *Define*
    - *Code*
    - *Test*

### *Gather Data*

In [157]:
# Data can be downloaded at following link

zip_file_url = 'https://cdn.spotle.ai/zip/Tweeter_Data_IN.csv.zip'

r = requests.get(zip_file_url)
z = zipfile.ZipFile(io.BytesIO(r.content))

z.extractall() # To extract folder at current directory

In [158]:
# Loading data

data = pd.read_csv('Tweeter_Data_IN.csv', header = 0)

In [159]:
# Some glimpse at data
data.sample(5)

,created_at,hashtags,favorite_count,id,lang,place,retweet_count,text,tweet_url,user_screen_name,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name.1,user_statuses_count
10360,Sat Mar 28 18:56:48 +0000 2020,NaN,1,1243975344673060000,en,"New Delhi, India",1,It's a shame that despite all appeals that the...,https://twitter.com/Girja_gsk/status/124397534...,Girja_gsk,Special Correspondent With The Tribune,125,164,99,1,"New Delhi, India",Girja Shankar Kaura,Girja_gsk,130
5755,Fri Mar 27 08:11:58 +0000 2020,Corona,3,1243450679827380000,en,"Noida, India",0,Respect unsung heroes of #Corona. https://t.co...,https://twitter.com/KumarGanesham/status/12434...,KumarGanesham,Proud volunteer of @myhomeindia | Social Worke...,1272,1388,605,2,"Deoghar, Jharkhand & New Delhi",Kumar Ganesham Jha,KumarGanesham,5207
34888,Sat May 02 12:50:45 +0000 2020,NaN,2,1256566797475720000,en,"Kanayannur, India",0,Covid19 map of India as on today. https://t.co...,https://twitter.com/kkaction/status/1256566797...,kkaction,Philosophically curious,11098,2238,21,174,India,Krishna Kumar,kkaction,10280
12213,Mon Mar 30 11:26:17 +0000 2020,NaN,0,1244586744021820000,en,"Noida, India",0,On what basis these morons call themselves jou...,https://twitter.com/jainpri2003/status/1244586...,jainpri2003,‡§µ‡§Ç‡§¶‡•á ‡§Æ‡§æ‡§§‡§∞‡§Æ‡•ç \n‡§≠‡§æ‡§∞‡§§...,41656,420,412,9,India,‡§∞‡§æ‡§∑‡•ç‡§ü‡•ç‡§∞ ‡§∏‡§∞‡•ç‡§µ‡§™‡•ç‡§∞‡§•‡§Æ,jainpri2003,50432
12342,Mon Mar 30 11:56:05 +0000 2020,Corona,0,1244594242585900000,en,"Sambalpur, India",1,"Apart from cases of #Corona affected people, s...",https://twitter.com/bimalpandia/status/1244594...,bimalpandia,"A development analyst by profession, a farmer ...",29967,1057,750,8,India,Bimal Pandia,bimalpandia,8875


### *Access*

#### Checking dataset

In [160]:
# Checking summary of data
# id column should be string rest is fine
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44179 entries, 0 to 44178
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   created_at             44179 non-null  object
 1   hashtags               17907 non-null  object
 2   favorite_count         44179 non-null  int64 
 3   id                     44179 non-null  int64 
 4   lang                   44179 non-null  object
 5   place                  44162 non-null  object
 6   retweet_count          44179 non-null  int64 
 7   text                   44179 non-null  object
 8   tweet_url              44179 non-null  object
 9   user_screen_name       44179 non-null  object
 10  user_description       37905 non-null  object
 11  user_favourites_count  44179 non-null  int64 
 12  user_followers_count   44179 non-null  int64 
 13  user_friends_count     44179 non-null  int64 
 14  user_listed_count      44179 non-null  int64 
 15  user_location      

In [161]:
# Checking numeric properties
data.describe()

,favorite_count,id,retweet_count,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_statuses_count
count,44179.000000,4.417900e+04,44179.000000,44179.000000,4.417900e+04,44179.000000,44179.000000,4.417900e+04
mean,8.688834,1.249746e+18,2.037122,15413.078906,6.076601e+03,908.844723,15.648951,1.830381e+04
std,690.196275,5.513385e+15,110.332251,42971.397214,1.494865e+05,5802.053827,86.748546,5.619829e+04
min,0.000000,1.242693e+18,0.000000,0.000000,0.000000e+00,0.000000,0.000000,1.000000e+00
25%,0.000000,1.244130e+18,0.000000,233.000000,3.400000e+01,98.000000,0.000000,3.990000e+02
50%,0.000000,1.250482e+18,0.000000,2192.000000,1.750000e+02,293.000000,0.000000,2.476000e+03
75%,1.000000,1.256266e+18,0.000000,11439.000000,8.890000e+02,843.000000,5.000000,1.330600e+04
max,144012.000000,1.258242e+18,22549.000000,638151.000000,1.827721e+07,316697.000000,5775.000000,2.117851e+06


In [162]:
# Checking column names
# Unnecassary Columns
# Column names are lower case and fine
print(data.columns)

Index(['created_at', 'hashtags', 'favorite_count', 'id', 'lang', 'place',
       'retweet_count', 'text', 'tweet_url', 'user_screen_name',
       'user_description', 'user_favourites_count', 'user_followers_count',
       'user_friends_count', 'user_listed_count', 'user_location', 'user_name',
       'user_screen_name.1', 'user_statuses_count'],
      dtype='object')


In [163]:
# Checking size of data
data.shape

(44179, 19)

In [164]:
# Checking missing data
data.isna().sum()

created_at                   0
hashtags                 26272
favorite_count               0
id                           0
lang                         0
place                       17
retweet_count                0
text                         0
tweet_url                    0
user_screen_name             0
user_description          6274
user_favourites_count        0
user_followers_count         0
user_friends_count           0
user_listed_count            0
user_location             6677
user_name                    3
user_screen_name.1           0
user_statuses_count          0
dtype: int64

In [165]:
# Ratio of missing values

print('Proportion of missing data:') 
print(data.isna().sum() / data.shape[0] * 100)

Proportion of missing data:
created_at                0.000000
hashtags                 59.467168
favorite_count            0.000000
id                        0.000000
lang                      0.000000
place                     0.038480
retweet_count             0.000000
text                      0.000000
tweet_url                 0.000000
user_screen_name          0.000000
user_description         14.201317
user_favourites_count     0.000000
user_followers_count      0.000000
user_friends_count        0.000000
user_listed_count         0.000000
user_location            15.113515
user_name                 0.006791
user_screen_name.1        0.000000
user_statuses_count       0.000000
dtype: float64


In [166]:
# We dont need hashtags in accordance to user so that can be seperated
# The standard deviations are high, means there are lot of zeroes in count columns which we dont need

# Relevant data with most information
data.query('retweet_count > 0 and user_favourites_count > 0 and user_friends_count > 0 and user_followers_count > 0').shape

(8752, 19)

In [167]:
# Checking duplicates
# No duplicates
data.duplicated().sum()

0

### *Clean Data*

#### *Collecting all the text as seperate document for further analysis*
#### *Collecting hashtags*

### *Text Processing*
### Text Processing Steps:
- Converting Upper or title case letters to lower case
- Removing Escape characters such as new line
- Removing Hyperlinks
- Removing Punctations
- Removing Numbers
- Removing stop words (a, an, the etc.)
- Removing profane words if neccessary
- Stemming or lemmatization

In [168]:
# Creating list of data
# Removing null values in list

temp_text = list(filter(lambda x : str(x) != 'nan', data.text))
temp_hashtags = list(filter(lambda x : str(x) != 'nan', data.hashtags))

#Converting to lowercase
temp_text = [" ".join(line.lower().strip().split()) for line in temp_text]
temp_hashtags = [" ".join(line.lower().strip().split()) for line in temp_hashtags]

print(len(temp_text))
print(len(temp_hashtags))

44179
17907


In [169]:
# Since there are multiple hashtags present in data and spliited by space, we have to seperate them
new_temp_hashtags = []
for line in temp_hashtags:
    new_temp_hashtags.extend(line.split())

new_temp_hashtags = ['#' + hashtag for hashtag in new_temp_hashtags]

# Removing Punctuations except @ as only one username will be present
line = line.translate(str.maketrans('', '', '!"$%&\'()*+,-./:;#<=>?[\\]^_`{|}~'))
    
# Removing Numbers
line = re.sub(r'\d+', '', line)

print(len(new_temp_hashtags)) 

60807


In [170]:
# Cleaning lines

# Not removing mentions or @usernames from tweets as tweet might become unreadable
# But removing @usernames if present at start as it indicates it is a retweet.

# Collects hastags used in tweets
hashtags = dict()
def removing_usernames_and_collecting_hashtags(line):
    data = line.split()
    if len(data) < 1:
        return line
    
    for item in data[1:] if data[0][0] == '@' else data: # Because after spliiting if userid present then it will always be at index 0 and will be key
        if item[0] == '#':
            if item in hashtags:
                hashtags[item] += 1
            else:
                hashtags[item] = 1
        elif item[0] == '@':
            data.remove(item)
    return " ".join(data)

def clean_line(line):
    line = line.lower().encode('ascii', errors = 'ignore').decode() # Converting to lower case will remove emojis [pure text]
    line = " ".join(line.strip().split()) # Split will automatically remove escape characters
    
    # Removing links
    line = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', '', line)
    
    # Collecting and removing usernames and hashtags
    line = removing_usernames_and_collecting_hashtags(line)
    
    # Removing Punctuations except @ as only one username will be present
    line = line.translate(str.maketrans('', '', '!"$%&\'()*+,-./:;#<=>?[\\]^_`{|}~'))
    
    # Removing Numbers
    line = re.sub(r'\d+', '', line)
    
    """
    # Removing emojis
    regrex_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030""]+", re.UNICODE)
    
    line = regrex_pattern.sub(r'',line)
    """

    # Removing stopwords
    stopword = stopwords.words('english')
    word_tokens = line.split()
    line_words = [word for word in word_tokens if word not in stopword]
    
    # Removing Profanity Words
    profanity = set()
    with open('Profanity.txt', mode = 'r') as text_profanity:
        reader = text_profanity.readlines()
        for word in reader:
            profanity.add(word.replace('\n', ''))
    line_words = [word for word in line_words if word not in profanity]
    
    # Stemming
    #line_words = [PorterStemmer().stem(word) for word in line_words]
    
    # Lemmatize
    line_words = [WordNetLemmatizer().lemmatize(word, pos = 'v') for word in line_words] # lemmatize parts of speech(pos)
    
    line = " ".join(line_words)
    return line    

In [171]:
tweets = []

for line in temp_text:
        if line == '\n':
            continue
        line = clean_line(line) # Cleaning line by calling function
        if line == '': # If empty string
            continue
        tweets.append(line)

In [172]:
print(tweets[:10])
print('\nTotal tweets : ', len(tweets))

['meditation mindfulness tip cop corona virus pandemicas people become anxious selfquarantine homeexperts recommend mindfulness meditation practice prove', 'contribute india fight corona gandhinagar gujarat', '@delhipolice sir please send team immediately sanitize build', 'sscufmmatkarona corona slow poison ufm instant killer thousands genuine aspirants @pmoindia', 'mata di doston ise corona ke tarah faila bhai please like share comment subscribe channel youtube pritamkumarmurari support voice', '@chintskap mum phd corona virus whatsapp university', '@funnyhammy dumbo amount count today emergency period countries announce billions dollars corona combat feku fool people like', 'first allow corona possessions stop kirana shop clearly nany bhakts listen modi anymore certainly yogi', '@tvmarathi kindly help us break chain corona', 'safe home corona busy infect people spread like wild fire home assure familyfriends amp everyone feel important safe amp sound remember corona bahar bulati hai 

In [173]:
# Adding hashtags to collected hashtags
for hashtag in new_temp_hashtags:
    if hashtag in hashtags:
        hashtags[hashtag] += 1
    else:
        hashtags[hashtag] = 1

In [174]:
# total hashtags

#print(hashtags)
print('yet_total_hashtags : ', len(hashtags))

yet_total_hashtags :  14754


#### Converting created_at timestamp to datetime format

In [175]:
# Converting datatime of created at in valid or datetime format

data.created_at = pd.to_datetime(data.created_at)
data.created_at.max(), data.created_at.min()

(Timestamp('2020-05-07 03:48:08+0000', tz='UTC'),
 Timestamp('2020-03-25 06:02:40+0000', tz='UTC'))

In [176]:
# Maintaining readable format
data.created_at = pd.to_datetime(data.created_at.dt.strftime("%d/%m/%y %H:%M:%S"))

In [177]:
data.describe()

,favorite_count,id,retweet_count,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_statuses_count
count,44179.000000,4.417900e+04,44179.000000,44179.000000,4.417900e+04,44179.000000,44179.000000,4.417900e+04
mean,8.688834,1.249746e+18,2.037122,15413.078906,6.076601e+03,908.844723,15.648951,1.830381e+04
std,690.196275,5.513385e+15,110.332251,42971.397214,1.494865e+05,5802.053827,86.748546,5.619829e+04
min,0.000000,1.242693e+18,0.000000,0.000000,0.000000e+00,0.000000,0.000000,1.000000e+00
25%,0.000000,1.244130e+18,0.000000,233.000000,3.400000e+01,98.000000,0.000000,3.990000e+02
50%,0.000000,1.250482e+18,0.000000,2192.000000,1.750000e+02,293.000000,0.000000,2.476000e+03
75%,1.000000,1.256266e+18,0.000000,11439.000000,8.890000e+02,843.000000,5.000000,1.330600e+04
max,144012.000000,1.258242e+18,22549.000000,638151.000000,1.827721e+07,316697.000000,5775.000000,2.117851e+06


In [178]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44179 entries, 0 to 44178
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   created_at             44179 non-null  datetime64[ns]
 1   hashtags               17907 non-null  object        
 2   favorite_count         44179 non-null  int64         
 3   id                     44179 non-null  int64         
 4   lang                   44179 non-null  object        
 5   place                  44162 non-null  object        
 6   retweet_count          44179 non-null  int64         
 7   text                   44179 non-null  object        
 8   tweet_url              44179 non-null  object        
 9   user_screen_name       44179 non-null  object        
 10  user_description       37905 non-null  object        
 11  user_favourites_count  44179 non-null  int64         
 12  user_followers_count   44179 non-null  int64         
 13  u

- ***Since we have collected Tweets and hashtags, we dont need them in dataframe anymore.***
- ***We dont need id column as data doesn't make difference.***

In [179]:
# Dropping unnecessary columns

data.drop(['hashtags', 'text', 'id'], inplace = True, axis = 1);
data.info();

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44179 entries, 0 to 44178
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   created_at             44179 non-null  datetime64[ns]
 1   favorite_count         44179 non-null  int64         
 2   lang                   44179 non-null  object        
 3   place                  44162 non-null  object        
 4   retweet_count          44179 non-null  int64         
 5   tweet_url              44179 non-null  object        
 6   user_screen_name       44179 non-null  object        
 7   user_description       37905 non-null  object        
 8   user_favourites_count  44179 non-null  int64         
 9   user_followers_count   44179 non-null  int64         
 10  user_friends_count     44179 non-null  int64         
 11  user_listed_count      44179 non-null  int64         
 12  user_location          37502 non-null  object        
 13  u

#### Now, removing null values

In [180]:
# Now removing data with null values and duplicates

data.duplicated().sum()
data.dropna(inplace = True)

In [181]:
# Info of final dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33991 entries, 0 to 44178
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   created_at             33991 non-null  datetime64[ns]
 1   favorite_count         33991 non-null  int64         
 2   lang                   33991 non-null  object        
 3   place                  33991 non-null  object        
 4   retweet_count          33991 non-null  int64         
 5   tweet_url              33991 non-null  object        
 6   user_screen_name       33991 non-null  object        
 7   user_description       33991 non-null  object        
 8   user_favourites_count  33991 non-null  int64         
 9   user_followers_count   33991 non-null  int64         
 10  user_friends_count     33991 non-null  int64         
 11  user_listed_count      33991 non-null  int64         
 12  user_location          33991 non-null  object        
 13  u

In [182]:
# sample data
data.sample(10)

,created_at,favorite_count,lang,place,retweet_count,tweet_url,user_screen_name,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name.1,user_statuses_count
42950,2020-06-05 07:15:39,0,en,"Hospet, India",0,https://twitter.com/veeranna_kambli/status/125...,veeranna_kambli,Do more of what makes u smileüòä. # Mechanica...,14034,69,465,0,"Bengaluru South, India",Veeranna Kambli,veeranna_kambli,438
32595,2020-01-05 11:36:36,0,en,"Guwahati, India",0,https://twitter.com/AnkurGit/status/1256185751...,AnkurGit,Student of Law/BALLB (First Year),32953,46,18,0,"Guwahati, Assam, IndiaüáÆüá≥",Ankur Deka,AnkurGit,5234
34007,2020-02-05 06:36:17,4,en,"Mumbai, India",1,https://twitter.com/lifeofapatrakar/status/125...,lifeofapatrakar,"Writer, Researcher, Education Reporter. TISS A...",5448,2287,1782,21,"Mumbai, India",Ankita Bhatkhande,lifeofapatrakar,3827
13713,2020-03-31 04:37:42,4,en,"Lucknow, India",1,https://twitter.com/abhishek6164/status/124484...,abhishek6164,"Editor @aajtak @IndiaToday, Bureau Chief UP.\n...",3786,7936,984,22,"Lucknow, India",kumar abhishek.,abhishek6164,4586
11809,2020-03-29 10:14:09,1,en,"Saidapet, India",1,https://twitter.com/muralwrites/status/1244206...,muralwrites,Public Relations practitioner - Freelance cont...,18337,4259,2962,108,"Chennai, India",murali,muralwrites,91795
31565,2020-04-21 03:46:15,4,en,"Chandigarh, India",1,https://twitter.com/drqayumiitk/status/1252443...,drqayumiitk,AGMUT Cadre‚Ä¢ Forest‚Ä¢ Wildlife‚Ä¢ Environme...,13410,10716,428,35,"Chandigarh, India","Dr Abdul Qayum, IFS",drqayumiitk,3847
30667,2020-04-20 10:14:38,1,en,"Chandigarh, India",0,https://twitter.com/navkamboj/status/125217885...,navkamboj,Tweets are personal & on a lighter note...,30898,166,773,1,Bharat,Uddta Punjab,navkamboj,31048
34533,2020-02-05 10:18:47,0,en,"Kolkata, India",0,https://twitter.com/BittuGurjant/status/125652...,BittuGurjant,WANT to become MP from DUM DUM (N) Kolkata on ...,109,19,24,0,"Dum Dum, Kolkata",GURRJANT SINGH MANUKA (ARJAN),BittuGurjant,1489
1063,2020-03-25 13:24:55,1,en,"Ranchi, India",0,https://twitter.com/kumaratul500/status/124280...,kumaratul500,go with flow..,139,8,228,0,"Ranchi, India",Atul Kumar Gupta,kumaratul500,320
36368,2020-03-05 06:18:54,0,en,"Bengaluru South, India",0,https://twitter.com/gn_vikram/status/125683057...,gn_vikram,I am not the doer. Things happen through me as...,625,54,341,3,Bangalore,Gajendra Vikram,gn_vikram,940


### *Remvoing columns we don't need*

- **tweet_url** : We will get nothing from it
- **user_description, user_name, user_screen_name1** : Personal Information
- **user_location** : entered by user, might be invalid and also personal and redundant.

In [183]:
data.drop(['tweet_url', 'user_description', 'user_location', 'user_name', 'user_screen_name.1'], axis = 1, inplace = True)

In [184]:
# Info on data
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33991 entries, 0 to 44178
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   created_at             33991 non-null  datetime64[ns]
 1   favorite_count         33991 non-null  int64         
 2   lang                   33991 non-null  object        
 3   place                  33991 non-null  object        
 4   retweet_count          33991 non-null  int64         
 5   user_screen_name       33991 non-null  object        
 6   user_favourites_count  33991 non-null  int64         
 7   user_followers_count   33991 non-null  int64         
 8   user_friends_count     33991 non-null  int64         
 9   user_listed_count      33991 non-null  int64         
 10  user_statuses_count    33991 non-null  int64         
dtypes: datetime64[ns](1), int64(7), object(3)
memory usage: 3.1+ MB


In [185]:
data.describe()

,favorite_count,retweet_count,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_statuses_count
count,33991.000000,33991.000000,33991.000000,3.399100e+04,33991.000000,33991.000000,3.399100e+04
mean,10.761054,2.446353,16972.715042,7.681606e+03,1028.718484,19.281163,1.953947e+04
std,786.783982,125.708909,45015.723386,1.703520e+05,6375.088496,96.536292,5.823652e+04
min,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,1.000000e+00
25%,0.000000,0.000000,444.000000,5.900000e+01,130.000000,0.000000,6.310000e+02
50%,0.000000,0.000000,3204.000000,2.710000e+02,359.000000,1.000000,3.459000e+03
75%,2.000000,0.000000,13675.500000,1.217000e+03,989.000000,7.000000,1.575100e+04
max,144012.000000,22549.000000,638151.000000,1.827721e+07,316697.000000,5775.000000,2.117851e+06


In [186]:
# Cleaned data
data.sample(10)

,created_at,favorite_count,lang,place,retweet_count,user_screen_name,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_statuses_count
10073,2020-03-28 17:35:04,0,en,"Basti, India",0,amitaimk,4913,631,1562,86,13055
23811,2020-04-16 18:36:44,0,en,"New Delhi, India",0,MohdIrshad_,7830,187,989,0,1853
4019,2020-03-26 14:51:33,0,en,"Rajendra Nagar, India",0,JithenderkBjym,12008,103,571,0,2909
19962,2020-04-14 10:01:10,4,en,"Maharashtra, India",1,FranchiseeMart,827,918,105,146,1751
34951,2020-02-05 13:39:22,11,en,"Egmore Nungambakkam, India",3,ranjith_tnpyc,2112,1033,1500,2,574
23039,2020-04-16 10:44:40,1,en,"Bihar, India",0,aamritanshuaman,90528,134,136,0,960
23736,2020-04-16 17:37:35,0,en,"Mhow, India",0,RsRmsc,2094,36,103,0,3883
31068,2020-04-20 15:32:10,0,en,"Ghaziabad, India",0,bhattachadilip,2290,488,1078,2,2383
15037,2020-03-31 12:38:58,3,en,"Jaipur, India",0,jameelwecan,8352,273,210,2,744
32663,2020-01-05 11:25:27,2,en,"Chengalpattu, India",0,maselvamani,61,66,332,0,212


In [187]:
# Checking if all data is from india or not as place only contains city or country or both
i = 0
for line in list(data.place):
    line = line.lower()
    if 'india' in line:
        i += 1
        
print(i) # Places with india in it

33025


#### ***Since, all data is from India removing the term India from places.***

In [188]:
# Cleaning place names
data.place = data.place.apply(clean_line)

# Removing country name, india if city present in the name
def remove_country(place):
    places = place.split()
    if len(places) > 1:
        if 'india' in places:
            places.remove('india')
    return " ".join(places)

data.place = data.place.apply(remove_country)

## Final Data

In [189]:
# Sample data
data.sample(10)

,created_at,favorite_count,lang,place,retweet_count,user_screen_name,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_statuses_count
33217,2020-01-05 18:11:58,0,en,trivandrum,0,Nmenon777,211327,3243,3724,6,101153
32297,2020-01-05 08:34:54,0,en,allahabad,0,Dr_Saurabh2803,4906,55,236,0,5711
17585,2020-01-04 12:46:05,1,en,andhra pradesh,0,NameisVarshi,3144,527,1558,0,5134
15843,2020-03-31 18:06:49,3,en,uttar pradesh,0,themohit_thakur,6864,719,203,1,1617
29359,2020-04-19 17:05:00,1,en,porbandar,0,thanki_harsh100,29922,644,803,0,7039
31038,2020-04-20 15:04:46,0,en,fort tondiarpet,0,perarivalan333,144777,1806,4942,0,139683
26062,2020-04-18 08:11:21,2,en,chhattisgarh,0,RamSajilaIYC,17547,1796,1975,1,28050
35724,2020-03-05 02:19:17,1,en,chandigarh,0,SrowSameer,1497,94,113,0,939
11184,2020-03-29 06:16:30,4,en,raichur,2,svmurthy,2154,1359,1196,138,70950
39838,2020-04-05 13:06:55,1,en,mumbai,1,pnandola,4958,733,1446,13,5931


In [190]:
# Tweets
tweets[:10]

['meditation mindfulness tip cop corona virus pandemicas people become anxious selfquarantine homeexperts recommend mindfulness meditation practice prove',
 'contribute india fight corona gandhinagar gujarat',
 '@delhipolice sir please send team immediately sanitize build',
 'sscufmmatkarona corona slow poison ufm instant killer thousands genuine aspirants @pmoindia',
 'mata di doston ise corona ke tarah faila bhai please like share comment subscribe channel youtube pritamkumarmurari support voice',
 '@chintskap mum phd corona virus whatsapp university',
 '@funnyhammy dumbo amount count today emergency period countries announce billions dollars corona combat feku fool people like',
 'first allow corona possessions stop kirana shop clearly nany bhakts listen modi anymore certainly yogi',
 '@tvmarathi kindly help us break chain corona',
 'safe home corona busy infect people spread like wild fire home assure familyfriends amp everyone feel important safe amp sound remember corona bahar bu

In [192]:
# Hashtags

list(hashtags.items())[:5]

[('#ssc_ufm_mat_karona', 2),
 ('#corona', 8665),
 ('#ufm', 2),
 ('#pritamkumarmurari', 2),
 ('#voice', 4)]

### Saving data to machine

In [203]:
# Storing Data 
data.to_csv('cleaned_data.csv', index = False)

# Storing Tweets
with open('cleaned_tweets.txt', 'w', encoding = 'utf8') as filehandle:
    filehandle.writelines("%s\n" % place for place in tweets)

# Storing hashtags
with open('hashtags.csv', 'w', encoding = 'utf8') as filehandle:
    writer = csv.writer(filehandle)
    for key, value in hashtags.items():
        #key = str.encode(key).decode('utf-8')
        #value = str.encode(value).decode('utf-8')
        writer.writerow([key, value])